_Does size matter? The effect of Instagram influencer account size on post sentiment and resulting marketing outcomes_

_Master's thesis by Thomas A. Frost_

# Part 1: Instagram Account profiles

This file is based on downloaded data files from the two data sources

## 02 - Setup

In [ ]:
library(tidyverse)
library(reshape)

## 03 - Data Import

In [ ]:
profiles_target <- read_csv("../data/fulldf-export.csv")
profiles_read <- read_tsv("../data/Instagram__Profiles.tsv")
profiles_error <- read_csv("../data/UserList_NoInstagramAccountFound.csv", col_names = FALSE)

colnames(profiles_target)[2] <- 'Username'
colnames(profiles_error)[1] <- 'Username'

profiles_error$found <- FALSE;
profiles_read$found <- TRUE;

# 04 - Matching

In [ ]:
profiles_target_short <- data.frame(profiles_target$Username, profiles_target$Instagram.Followers, profiles_target$size)
colnames(profiles_target_short) <- c("Username", "followers_old", "size_old")

profiles <- merge_recurse(list(profiles_target_short, profiles_read, profiles_error), by = "Username")
colnames(profiles)[8] <- 'followers_new'

## 05 - Descriptive

In [ ]:
summary(profiles$found)
summary(profiles$is_private)

## 06 - change in account size

In [ ]:
getsize <- function(followers) {
  result <- case_when(
    (followers <= 1000) ~ NA,
    (followers > 1000 && followers <= 10000) ~ "nano",
    (followers > 10000 && followers <= 50000) ~ "mikro",
    (followers > 50000 && followers <= 500000) ~ "midtier",
    (followers > 500000 && followers <= 1000000) ~ "makro",
    (followers > 1000000) ~ "mega"
  )
  return(result)
}

profiles$size_new <- unlist(map(profiles$followers_new, getsize))

profiles$size_new <- as.factor(profiles$size_new)

table(profiles$size_old, profiles$size_new, useNA = "ifany")

In [ ]:
write_csv(profiles, "../data/profiles_v2.csv")